# Fetch Results from DB

In [16]:
from dataclasses import dataclass, field
import math
import os
import random
import time
from typing import List, Dict

from collections import Counter
import csv

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psycopg2
from scipy.interpolate import interp1d

In [17]:
# handle db related stuff

# turn the above into a dictionary using the values
db_params = {
    "dbname": "decision_trees",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",
    "port": 5432
}

column_names = [
    "name", 
    "iterations", 
    "f1_score", 
    "feature_limit", 
    "feature_table_entries", 
    "max_depth", 
    "num_flows", 
    "num_partitions", 
    "total_features", 
    "tree_table_entries"
]

In [18]:
# table names for each of our datasets

iscxvpn2016_baseline = "sigcomm_baseline_iscxvpn2016_pcaps0_f10"
iscxvpn2016_hypermapper = "hypermapper_iscxvpn2016_pcaps0_f10_bayesian_optimization"

def get_dataset_entries(baseline_table, hypermapper_table):
    # open a connection with the db
    connection = psycopg2.connect(**db_params)
    cursor = connection.cursor()
    # select all entries from baseline_table
    cursor.execute(f"SELECT * FROM {baseline_table}")
    baseline_entries = cursor.fetchall()
    # select all entries from hypermapper_table
    cursor.execute(f"SELECT * FROM {hypermapper_table}")
    hypermapper_entries = cursor.fetchall()
    # combine the two lists
    entries = baseline_entries + hypermapper_entries
    # close the connection
    cursor.close()
    connection.close()
    return entries

iscxvpn2016_entries = get_dataset_entries(iscxvpn2016_baseline, iscxvpn2016_hypermapper)

# get number of entries in each dataset
print(f"Number of entries in iscxvpn2016: {len(iscxvpn2016_entries)}")

Number of entries in iscxvpn2016: 3429


In [19]:
# convert these entries into a pandas dataframe
iscxvpn2016_df = pd.DataFrame(iscxvpn2016_entries, columns=column_names)

# extend each dataframe to include tcam_entries column = feature_table_entries + tree_table_entries
iscxvpn2016_df["tcam_entries"] = iscxvpn2016_df["feature_table_entries"] + iscxvpn2016_df["tree_table_entries"]

In [20]:
# extend each dataframe to include memory_bits column = tcam_entries * 32
iscxvpn2016_df['memory_bits'] = iscxvpn2016_df.apply(
    lambda x: x['total_features'] * x['num_flows'] * 32 if x['name'] in ['leo', 'netbeacon'] else x['feature_limit'] * x['num_flows'] * 32,
    axis=1
)

# extend each dataframe to include memory percentage usage by dividing memory_bits by 120Mb
iscxvpn2016_df['memory_perc'] = round(iscxvpn2016_df['memory_bits'] / (120 * 1024 * 1024) * 100, 2)

## Save top-level dataframes

In [ ]:
# save the dataframes to csv files with same names as dataframes
top_level_path = "csv/top-level-dataframes"
if not os.path.exists(top_level_path):
    os.makedirs(top_level_path)

iscxvpn2016_df.to_csv(os.path.join(top_level_path, "iscxvpn2016_df.csv"), index=False)

# E2E Pareto Plots

In [22]:
# show all unique num_flows in each dataset
print(f"Unique num_flows in iscxvpn2016: {iscxvpn2016_df['num_flows'].unique().tolist()}")

# get the union list of all unique num_flows, exclude 0 and then sort it
unique_num_flows = sorted(list(set(
    iscxvpn2016_df['num_flows'].unique().tolist()
)))
unique_num_flows = [x for x in unique_num_flows if x > 0 and x <= 1000000]

print(f"\nOverall Unique num_flows: {unique_num_flows}")

Unique num_flows in iscxvpn2016: [1966080, 983040, 786432, 491520, 393216, 294912, 1769472, 589824, 262144, 1572864, 196608, 1376256, 1179648, 65536, 2000000, 0]

Overall Unique num_flows: [65536, 196608, 262144, 294912, 393216, 491520, 589824, 786432, 983040]


In [23]:
LIST_OF_NUM_FLOWS = [
    65536, 98304, 196608, 262144, 294912, 393216,
    491520, 589824, 688128, 786432, 884736, 983040
]

def extend_pareto(this_dataframe: pd.DataFrame, baseline_name: str) -> pd.DataFrame:
    this_dataframe = this_dataframe.copy()

    # -------------------------
    # 1) Fill step: ensure all flow counts exist
    # -------------------------
    missing = sorted(set(LIST_OF_NUM_FLOWS) - set(this_dataframe["num_flows"].values))
    if missing:
        fill_rows = pd.DataFrame([{
            "name": baseline_name,
            "iterations": 0,
            "f1_score": 0,
            "feature_limit": 0,
            "feature_table_entries": 0,
            "max_depth": 0,
            "num_flows": nf,
            "num_partitions": 0,
            "total_features": 0,
            "tree_table_entries": 0,
        } for nf in missing])

        this_dataframe = pd.concat([this_dataframe, fill_rows], ignore_index=True)

    # sort by num_flows
    this_dataframe = this_dataframe.sort_values(by="num_flows").reset_index(drop=True)

    # start with last flow count
    last_nf = LIST_OF_NUM_FLOWS[-1]
    current_row = this_dataframe.loc[this_dataframe["num_flows"] == last_nf]
    # (assumes at least one row exists after fill step)
    current_f1_score = float(current_row["f1_score"].iloc[0])
    current_model_config = current_row.copy()

    # iterate from high -> low
    for i, num_flows in enumerate(reversed(LIST_OF_NUM_FLOWS)):
        if i == 0:
            continue

        row_at_nf = this_dataframe.loc[this_dataframe["num_flows"] == num_flows]
        f1_at_nf = float(row_at_nf["f1_score"].iloc[0])

        if f1_at_nf < current_f1_score:
            # drop the inferior row
            this_dataframe = this_dataframe.loc[this_dataframe["num_flows"] != num_flows].copy()

            # create replacement row from current_model_config, but with updated num_flows
            row_to_copy = current_model_config.copy()
            row_to_copy.loc[:, "num_flows"] = num_flows  # keep all other columns the same

            # concat replacement
            this_dataframe = pd.concat([this_dataframe, row_to_copy], ignore_index=True)
        else:
            current_f1_score = f1_at_nf
            current_model_config = row_at_nf.copy()

    # final sort
    this_dataframe = this_dataframe.sort_values(by="num_flows").reset_index(drop=True)
    return this_dataframe


def pareto_processor(
    this_dataframe: pd.DataFrame,
    name: str = "this_dataframe",
    min_flows: int = 1,
    max_flows: int = 1_000_000,
    pareto: bool = True,
    verbose: bool = False,
) -> pd.DataFrame:

    # separate results from each model
    netbeacon_this_dataframe = this_dataframe[this_dataframe["name"] == "netbeacon"]
    leo_this_dataframe = this_dataframe[this_dataframe["name"] == "leo"]
    cap_this_dataframe = this_dataframe[this_dataframe["name"] == "cap"]

    if verbose:
        print(f"\n[Before] Number of entries in netbeacon_{name}: {len(netbeacon_this_dataframe)}")
        print(f"[Before] Number of entries in leo_{name}: {len(leo_this_dataframe)}")
        print(f"[Before] Number of entries in cap_{name}: {len(cap_this_dataframe)}")

    # for each model: for each num_flows pick best f1_score
    pareto_netbeacon_this_dataframe = netbeacon_this_dataframe.loc[
        netbeacon_this_dataframe.groupby("num_flows")["f1_score"].idxmax()
    ]
    pareto_leo_this_dataframe = leo_this_dataframe.loc[
        leo_this_dataframe.groupby("num_flows")["f1_score"].idxmax()
    ]
    pareto_cap_this_dataframe = cap_this_dataframe.loc[
        cap_this_dataframe.groupby("num_flows")["f1_score"].idxmax()
    ]

    if pareto:
        pareto_netbeacon_this_dataframe = extend_pareto(pareto_netbeacon_this_dataframe, "netbeacon")
        pareto_leo_this_dataframe = extend_pareto(pareto_leo_this_dataframe, "leo")
        pareto_cap_this_dataframe = extend_pareto(pareto_cap_this_dataframe, "cap")

    if verbose:
        print(f"\n[After] Number of entries in netbeacon_{name}: {len(pareto_netbeacon_this_dataframe)}")
        print(f"[After] Number of entries in leo_{name}: {len(pareto_leo_this_dataframe)}")
        print(f"[After] Number of entries in cap_{name}: {len(pareto_cap_this_dataframe)}")

    # combine
    pareto_this_dataframe = pd.concat(
        [pareto_netbeacon_this_dataframe, pareto_leo_this_dataframe, pareto_cap_this_dataframe],
        axis=0,
        ignore_index=True,
    )

    # filter by flow range
    pareto_this_dataframe = pareto_this_dataframe[
        (pareto_this_dataframe["num_flows"] >= min_flows) &
        (pareto_this_dataframe["num_flows"] <= max_flows)
    ].reset_index(drop=True)

    return pareto_this_dataframe


In [24]:
pareto_iscxvpn2016_df = pareto_processor(iscxvpn2016_df, "iscxvpn2016")

In [25]:
# show rows with 'cap' in them only
pareto_iscxvpn2016_df[pareto_iscxvpn2016_df['name'] == 'cap']

pareto_iscxvpn2016_df

,name,iterations,f1_score,feature_limit,feature_table_entries,max_depth,num_flows,num_partitions,total_features,tree_table_entries,tcam_entries,memory_bits,memory_perc
0,netbeacon,0,0.780000,0,0,15,65536,0,6,3776,3776.0,37748736.0,30.0
1,netbeacon,0,0.780000,0,0,15,98304,0,6,3776,3776.0,37748736.0,30.0
2,netbeacon,0,0.780000,0,0,15,196608,0,6,3776,3776.0,37748736.0,30.0
3,netbeacon,0,0.740000,0,0,9,262144,0,6,1848,1848.0,75497472.0,60.0
4,netbeacon,0,0.740000,0,0,9,294912,0,6,1848,1848.0,75497472.0,60.0
5,netbeacon,0,0.740000,0,0,9,393216,0,6,1848,1848.0,75497472.0,60.0
6,netbeacon,0,0.570000,0,0,13,491520,0,4,1713,1713.0,62914560.0,50.0
7,netbeacon,0,0.300000,0,0,15,589824,0,3,514,514.0,75497472.0,60.0
8,netbeacon,0,0.300000,0,0,15,688128,0,3,514,514.0,75497472.0,60.0
9,netbeacon,0,0.300000,0,0,15,786432,0,3,514,514.0,75497472.0,60.0


In [ ]:
# save the dataframes to csv files with same names as dataframes
e2e_pareto_path = "csv/e2e-pareto-dataframes"
if not os.path.exists(e2e_pareto_path):
    os.makedirs(e2e_pareto_path)

pareto_iscxvpn2016_df.to_csv(os.path.join(e2e_pareto_path, "pareto_iscxvpn2016_df.csv"), index=False)